In [ ]:
function BSGM_biclust(Y; init = Y, v0 = 1e-1, sigma = 1, seed = 1, iter = 100, k1 = 2, k2 = 2)
    
    function update(Y,Theta,Mu,sigma,v0; transposed = false)
        n = size(Y,1); k = size(Mu,1);
        diff_sq = reshape(sum((repeat(Theta,outer = [k,1]) - repeat(Mu,inner = [n,1])).^2,2),n,k);
        Q = exp.(-diff_sq/(v0*sigma^2));
        Q = Q./ sum(Q,2);
        R = Diagonal(1./sum(Q,1)[:]) * Q';
        Theta = (eye(n) + (eye(n) - Q * R)/v0)\Y;
        Mu = R * Theta;
        if transposed == false return Theta, Mu, Q
        else return Theta', Mu', Q'
        end
    end
    
    # initialize
    srand(seed)
    n1,n2 = size(Y);
    Theta = init + 0.0 * randn(n1,n2);
    S1 = randperm(n1)[1:k1];
    S2 = randperm(n2)[1:k2];
    Mu1 = init[S1,:] + 0.0 * randn(k1,n2);
    Mu2 = init[:,S2] + 0.0 * randn(n1,k2);
    Q1 = randn(n1,k1);
    Q2 = randn(k2,n2);
    
    # loop start
    for i = 1:iter
        Theta, Mu1, Q1 = update(Y,Theta,Mu1,sigma,v0);
        Theta, Mu2, Q2 = update(Y',Theta',Mu2',sigma,v0, transposed = true);
    end
    
    # get a final estimate
    A = (Q2'*Diagonal(1./sum(Q2,2)[:]) * Q2);
    B = (Q1*Diagonal(1./sum(Q1,1)[:]) * Q1')
    g1 = Graph(A .> 1e-3)
    g2 = Graph(B .> 1e-3)
    c1 = connected_components(g1);
    c2 = connected_components(g2);
    ind = c2[1];
    for i = 2:length(c2)
        ind = [ind;c2[i]];
    end
    Final = (B * Theta)[ind,:];
    
    return Final, ind, A, B, Theta
end